In [2]:
#Import Dependencies
import pandas as pd
import numpy as np

In [6]:
# path to csv and data frames
school_data_csv = "Resources/schools_complete.csv"
student_data_csv = "Resources/students_complete.csv"

school_data = pd.read_csv(school_data_csv) 
student_data = pd.read_csv(student_data_csv) 

#merge into single data set
school_data_complete = pd.merge(student_data, school_data, how='left', on=["school_name", "school_name"])
school_data_complete


,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635
...,...,...,...,...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12th,Thomas High School,99,90,14,Charter,1635,1043130
39166,39166,Dawn Bell,F,10th,Thomas High School,95,70,14,Charter,1635,1043130
39167,39167,Rebecca Tanner,F,9th,Thomas High School,73,84,14,Charter,1635,1043130
39168,39168,Desiree Kidd,F,10th,Thomas High School,99,90,14,Charter,1635,1043130


In [33]:
#create district summary use .nunique() for total schools and students
total_schools = school_data_complete['School ID'].nunique() 
total_students = school_data_complete['Student ID'].nunique() 

#budget is an integer so simply sum it, use school_data not complete use float for the decimal
total_dist_budget = float(school_data['budget'].sum())  

#average scores take mean
#use round(data, number of decimal places) to clean up data
dist_math_score = float(round(school_data_complete['math_score'].mean(),2))
dist_reading_score = float(round(school_data_complete['reading_score'].mean(),2))

#overall pass rate is both averages summed and divided by two
dist_pass_rate = round((dist_math_score + dist_reading_score) / 2, 2)  

#calculate percentage of students with passing scores...get total number passing first then divide by total students need to put float(len())
dist_num_math = float(len(school_data_complete.loc[school_data_complete['math_score'] >= 70]))
dist_num_reading = float(len(school_data_complete.loc[school_data_complete['reading_score'] >= 70]))
dist_perc_math = float(round((dist_num_math / total_students) * 100, 2))
dist_perc_reading = float(round((dist_num_reading / total_students) * 100, 2)) 



In [35]:
#put all created values above into a dataframe use pd.DataFrame({"Name":variable,"Name2":variable... })
#https://www.geeksforgeeks.org/different-ways-to-create-pandas-dataframe/  ---> add index=[0]
dist_summary_df = pd.DataFrame({"Total Schools":total_schools, "Total Students":total_students, 
                                "Total Budget":total_dist_budget, "Average Math Score":dist_math_score,
                                "Average Reading Score":dist_reading_score, "Percent Passing Math":dist_perc_math,
                                "Percent Passing Reading":dist_perc_reading, "Overall Passing Rate (Percent)":dist_pass_rate}, index=[0])  
dist_summary_df

#clean up data with fomatting for budget and percentages


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Overall Passing Rate (Percent)
0,15,39170,24649428.0,78.99,81.88,74.98,85.81,80.44


In [87]:
#School Summary (DO NOT OVERWRITE PREVIOUSLY CREATED DATAFRAMES)
#FORMAT FOR VARIABLES: school_summ_students 
#do a groupby('school_name') to start the process

school_summ_group = school_data_complete.groupby(['school_name'])

#school type 
#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.groupby.GroupBy.first.html .first() returns series
school_summ_type = school_summ_group['type'].first() 

#total students use ['Student ID'] .size() this returns number of rows by group 
#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.groupby.GroupBy.size.html
school_summ_students = school_summ_group.size() 

#Total budget
school_summ_budget = school_summ_group['budget'].first() 

#per student budget
school_summ_budget_student = school_summ_budget / school_summ_students

#average scores in math and reading use round( , 2)
school_math_score = round(school_summ_group['math_score'].mean(), 2) 
school_reading_score = round(school_summ_group['reading_score'].mean(), 2)

#number passing and percentage passing math and reading refer to district df above need to groupby name in congunction with all scores
school_num_math = school_data_complete[school_data_complete['math_score'] >= 70].groupby(['school_name']).size()
school_perc_math = round((school_num_math / school_summ_students) * 100, 2) 
school_num_reading = school_data_complete[school_data_complete['reading_score'] >= 70].groupby(['school_name']).size()
school_perc_reading = round((school_num_reading / school_summ_students) * 100, 2) 
 
#school overall passing rate is average of school_perc_math and perc_reading respectively
school_overall_pass = round((school_perc_math + school_perc_reading) / 2, 2)



school_name
Bailey High School       74.31
Cabrera High School      95.58
Figueroa High School     73.36
Ford High School         73.81
Griffin High School      95.26
Hernandez High School    73.81
Holden High School       94.38
Huang High School        73.50
Johnson High School      73.64
Pena High School         95.27
Rodriguez High School    73.30
Shelton High School      94.86
Thomas High School       95.29
Wilson High School       95.21
Wright High School       94.97
dtype: float64

In [94]:
#put all the school data into a dataframe
#bug fixing: do not include the school_name in the dataframe it is already grouped by this
#bug fixing: do not forget the , after the last set in the df 
#bug fixing: no need to include index=[0] 
school_summary_df = pd.DataFrame({"School Type":school_summ_type,
                                 "Total Students":school_summ_students, "Total School Budget":school_summ_budget,
                                 "Per Student Budget":school_summ_budget_student, "Average Math Score":school_math_score,
                                 "Average Reading Score":school_reading_score, "Percent Passing Math":school_perc_math,
                                 "Pecent Passing Reading":school_perc_reading, "Overall Passing Rate":school_overall_pass,})
school_summary_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,Percent Passing Math,Pecent Passing Reading,Overall Passing Rate
school_name,,,,,,,,,
Bailey High School,District,4976,3124928,628.0,77.05,81.03,66.68,81.93,74.31
Cabrera High School,Charter,1858,1081356,582.0,83.06,83.98,94.13,97.04,95.58
Figueroa High School,District,2949,1884411,639.0,76.71,81.16,65.99,80.74,73.36
Ford High School,District,2739,1763916,644.0,77.10,80.75,68.31,79.30,73.81
Griffin High School,Charter,1468,917500,625.0,83.35,83.82,93.39,97.14,95.26
Hernandez High School,District,4635,3022020,652.0,77.29,80.93,66.75,80.86,73.81
Holden High School,Charter,427,248087,581.0,83.80,83.81,92.51,96.25,94.38
Huang High School,District,2917,1910635,655.0,76.63,81.18,65.68,81.32,73.50
Johnson High School,District,4761,3094650,650.0,77.07,80.97,66.06,81.22,73.64


In [96]:
#Top Performing Schools by passing rate: for this you use .sort_values function by the 'Overall Passing Rate' index and asending=false 
top_schools_df = school_summary_df.sort_values('Overall Passing Rate', ascending=False) 
top_schools_df.head(5) 

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,Percent Passing Math,Pecent Passing Reading,Overall Passing Rate
school_name,,,,,,,,,
Cabrera High School,Charter,1858,1081356,582.0,83.06,83.98,94.13,97.04,95.58
Thomas High School,Charter,1635,1043130,638.0,83.42,83.85,93.27,97.31,95.29
Pena High School,Charter,962,585858,609.0,83.84,84.04,94.59,95.95,95.27
Griffin High School,Charter,1468,917500,625.0,83.35,83.82,93.39,97.14,95.26
Wilson High School,Charter,2283,1319574,578.0,83.27,83.99,93.87,96.54,95.21


In [97]:
#Bottom Performing Schools is the same as above but ascending is true
bottom_schools_df = school_summary_df.sort_values('Overall Passing Rate', ascending=True) 
bottom_schools_df.head(5) 

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,Percent Passing Math,Pecent Passing Reading,Overall Passing Rate
school_name,,,,,,,,,
Rodriguez High School,District,3999,2547363,637.0,76.84,80.74,66.37,80.22,73.30
Figueroa High School,District,2949,1884411,639.0,76.71,81.16,65.99,80.74,73.36
Huang High School,District,2917,1910635,655.0,76.63,81.18,65.68,81.32,73.50
Johnson High School,District,4761,3094650,650.0,77.07,80.97,66.06,81.22,73.64
Ford High School,District,2739,1763916,644.0,77.10,80.75,68.31,79.30,73.81
